In [16]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import tensorflow.keras.layers as layers

In [17]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(r'C:\Users\Administrator\Desktop\Dataset\PetImages\train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 19998 images belonging to 2 classes.


In [18]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(r'C:\Users\Administrator\Desktop\Dataset\PetImages\test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 4999 images belonging to 2 classes.


In [19]:
class VGG11(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.conv_layer1 = tf.keras.Sequential([
            layers.Conv2D(filters=64,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer1 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.conv_layer2 = tf.keras.Sequential([
            layers.Conv2D(filters=128,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer2 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        

        self.conv_layer3 = tf.keras.Sequential([
            layers.Conv2D(filters=256,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.conv_layer4 = tf.keras.Sequential([
            layers.Conv2D(filters=256,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer3 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)


        self.conv_layer5 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.conv_layer6 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer4 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)

        self.conv_layer7 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.conv_layer8 = tf.keras.Sequential([
            layers.Conv2D(filters=512,kernel_size=3,strides=1,padding="same"),
            layers.BatchNormalization(),
            layers.ReLU()
        ])
        self.sub_sampling_layer5 = layers.MaxPooling2D(pool_size=(2, 2),strides=2)
        
        self.flatten = layers.Flatten() 

        self.fc_layer1 = tf.keras.Sequential([
        layers.Dense(4096),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.fc_layer2 = tf.keras.Sequential([
        layers.Dense(4096),
        layers.ReLU(),
        layers.Dropout(0.5)
        ])
        self.output_layer = tf.keras.Sequential([
        layers.Dense(2),
        layers.Activation('sigmoid')
        ])

    def call(self,x):
        x = self.conv_layer1(x)
        x = self.sub_sampling_layer1(x)
        x = self.conv_layer2(x)
        x = self.sub_sampling_layer2(x)
        x = self.conv_layer3(x)
        x = self.conv_layer4(x)
        x = self.sub_sampling_layer3(x)
        x = self.conv_layer5(x)
        x = self.conv_layer6(x)
        x = self.sub_sampling_layer4(x)
        x = self.conv_layer7(x)
        x = self.conv_layer8(x)
        x = self.sub_sampling_layer5(x)
        x = self.flatten(x)
        x = self.fc_layer1(x)
        x = self.fc_layer2(x)
        x = self.output_layer(x)
        
        return x

In [20]:
model = VGG11()

In [28]:
model.build(input_shape=(None,224,224,3))

ValueError: Exception encountered when calling layer 'sequential_1' (type Sequential).

Input 0 of layer "conv2d_2" is incompatible with the layer: expected axis -1 of input shape to have value 224, but received input with shape (None, 224, 224, 3)

Call arguments received by layer 'sequential_1' (type Sequential):
  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)
  • training=None
  • mask=None

In [22]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.